In [ ]:
import json
import requests_with_caching
def get_movies_from_tastedive(movieName):
    baseurl="https://tastedive.com/api/similar"
    params_d = {}
    params_d["q"]= movieName

    params_d["type"]= "movies"
    params_d["limit"] = "5"
    
    resp = requests_with_caching.get(baseurl, params=params_d)
    
    respDic = resp.json()
    return respDic 
def extract_movie_titles(resp):
    x=resp['Similar']['Results']
    x=[y['Name'] for y in x]
    return(x)
def get_related_titles(list_of_movie_title):
    new_list = list()
    for title in list_of_movie_title:
        a = get_movies_from_tastedive(title)
        b = extract_movie_titles(a)
        for movie in b:
            if movie not in new_list:
                new_list.append(movie)
    return new_list
def get_movie_data(movie_name):
    parameters = {'t': movie_name, 'r': 'json'}
    omdbapi_response = requests_with_caching.get('http://www.omdbapi.com/', params=parameters)
    a = json.loads(omdbapi_response.text)
    return a

def get_movie_rating(movieNameJson):
    strRanting=""
    for typeRantingList in movieNameJson["Ratings"]:
        if typeRantingList["Source"]== "Rotten Tomatoes":
            strRanting = typeRantingList["Value"]
    if strRanting != "":
        ranting = int(strRanting[:2])
    else: ranting = 0
    return ranting
def get_sorted_recommendations(listMovieTitle):
    listMovie= get_related_titles(listMovieTitle)
    listMovie= sorted(listMovie, key = lambda movieName: (get_movie_rating(get_movie_data(movieName)), movieName), reverse=True)
    
    return listMovie
get_sorted_recommendations(["Bridesmaids", "Sherlock Holmes"])

